In [1]:
import os
import cv2
import json
import torch
import pprint
import numpy as np
from tqdm import tqdm
from loguru import logger
from tabulate import tabulate

from pose.utils import collate_fn, geodesic_distance, relative_pose_error, aggregate_metrics, recall_object, project_points

In [2]:
LM_id2name_dict = {
    1: 'ape',
    2: 'benchvise',
    4: 'camera',
    5: 'can',
    6: 'cat',
    8: 'driller',
    9: 'duck',
    10: 'eggbox',
    11: 'glue',
    12: 'holepuncher',
    13: 'iron',
    14: 'lamp',
    15: 'phone',
}

In [3]:
with open("data/pairs/LINEMOD-test.json") as f:
    dir_list = json.load(f)
len(dir_list)

13

In [4]:
ROOT_DIR = 'data/LM_dataset/'

In [5]:
res_table = []

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = 'cpu'

num_sample = 400

net = torch.load('./weights/linemod-6d-400-2024-01-06-00-40-26-1.7288.pth').to(device)

net.eval()

Mkpts_Reg_Model(
  (embedding): Embedding()
  (transformerlayer): TransformerEncoderLayer(
    (self_attn): MultiheadAttention(
      (out_proj): NonDynamicallyQuantizableLinear(in_features=76, out_features=76, bias=True)
    )
    (linear1): Linear(in_features=76, out_features=2048, bias=True)
    (dropout): Dropout(p=0.1, inplace=False)
    (linear2): Linear(in_features=2048, out_features=76, bias=True)
    (norm1): LayerNorm((76,), eps=1e-05, elementwise_affine=True)
    (norm2): LayerNorm((76,), eps=1e-05, elementwise_affine=True)
    (dropout1): Dropout(p=0.1, inplace=False)
    (dropout2): Dropout(p=0.1, inplace=False)
  )
  (transformer): TransformerEncoder(
    (layers): ModuleList(
      (0-3): 4 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=76, out_features=76, bias=True)
        )
        (linear1): Linear(in_features=76, out_features=2048, bias=True)
        (dropout): Dropout(p=0.1, inp

In [7]:
for label_idx, test_dict in enumerate(dir_list):
    logger.info(f"LINEMOD: {label_idx + 1}/{len(dir_list)}")
    metrics = dict()
    metrics.update({'R_errs':[], 't_errs':[], 'inliers':[], "identifiers":[]})
    sample_data = dir_list[label_idx]["0"][0]
    label = sample_data.split("/")[0]
    name = label.split("-")[1]
    dir_name = os.path.dirname(sample_data)
    FULL_ROOT_DIR = os.path.join(ROOT_DIR, dir_name)
    recall_image, all_image = 0, 0
    for rotation_key, rotation_list in zip(test_dict.keys(), test_dict.values()):
        for pair_idx, pair_name in enumerate(tqdm(rotation_list)):
            all_image = all_image + 1
            base_name = os.path.basename(pair_name)
            idx0_name = base_name.split("-")[0]
            idx1_name = base_name.split("-")[1]
            image0_name = os.path.join(FULL_ROOT_DIR, idx0_name)
            image1_name = os.path.join(FULL_ROOT_DIR.replace("color", "color_full"), idx1_name)

            K0_path = image0_name.replace("color", "intrin_ba").replace("png", "txt")
            K1_path = image1_name.replace("color_full", "intrin").replace("png", "txt")
            K0 = np.loadtxt(K0_path)
            K1 = np.loadtxt(K1_path)

            pose0_path = image0_name.replace("color", "poses_ba").replace("png", "txt")
            pose1_path = image1_name.replace("color_full", "poses_ba").replace("png", "txt")
            pose0 = np.loadtxt(pose0_path)
            pose1 = np.loadtxt(pose1_path)
            if pose0.shape[0] == 3:
                pose0 = np.concatenate([pose0, np.array([[0, 0, 0, 1]])], axis=0)
                pose1 = np.concatenate([pose1, np.array([[0, 0, 0, 1]])], axis=0)

            points_file_path = os.path.join('d:/git_project/POPE/data/LM_dataset-points/', pair_name.split("/")[0])
            pre_bbox_path = os.path.join(points_file_path, "pre_bbox")
            mkpts0_path = os.path.join(points_file_path, "mkpts0")
            mkpts1_path = os.path.join(points_file_path, "mkpts1")
            pre_K_path = os.path.join(points_file_path, "pre_K")
            points_name = pair_name.split("/")[-1]
            pre_bbox_path = os.path.join(pre_bbox_path, f'{points_name}.txt')
            mkpts0_path = os.path.join(mkpts0_path, f'{points_name}.txt')
            mkpts1_path = os.path.join(mkpts1_path, f'{points_name}.txt')
            pre_K_path = os.path.join(pre_K_path, f'{points_name}.txt')

            if not os.path.exists(pre_bbox_path):
                continue
            pre_bbox = np.loadtxt(pre_bbox_path)
            mkpts0 = np.loadtxt(mkpts0_path)
            mkpts1 = np.loadtxt(mkpts1_path)
            pre_K = np.loadtxt(pre_K_path)

            if mkpts0.shape[0] > num_sample:
                rand_idx = np.random.choice(mkpts0.shape[0], num_sample, replace=False)
                mkpts0 = mkpts0[rand_idx]
                mkpts1 = mkpts1[rand_idx]
            else:
                mkpts0 = np.concatenate([mkpts0, np.zeros((num_sample - mkpts0.shape[0], 2))], axis=0)
                mkpts1 = np.concatenate([mkpts1, np.zeros((num_sample - mkpts1.shape[0], 2))], axis=0)

            _3d_bbox = np.loadtxt(f"{os.path.join(ROOT_DIR, label)}/box3d_corners.txt")
            bbox_pts_3d, _ = project_points(_3d_bbox, pose1[:3, :4], K1)
            bbox_pts_3d = bbox_pts_3d.astype(np.int32)
            x0, y0, w, h = cv2.boundingRect(bbox_pts_3d)
            x1, y1 = x0 + w, y0 + h
            gt_bbox = np.array([x0, y0, x1, y1])
            is_recalled = recall_object(pre_bbox, gt_bbox)
            recall_image = recall_image + int(is_recalled > 0.5)

            batch_mkpts0 = torch.from_numpy(mkpts0).unsqueeze(0).float().to(device)
            batch_mkpts1 = torch.from_numpy(mkpts1).unsqueeze(0).float().to(device)
            pre_t, pre_rot = net(batch_mkpts0, batch_mkpts1)
            # pre_t = pre_t.cpu()
            # pre_rot = pre_rot.cpu()

            # batch_pose0 = torch.from_numpy(pose0).unsqueeze(0).float().to(device)
            # batch_pose1 = torch.from_numpy(pose1).unsqueeze(0).float().to(device)
            # batch_relative_pose = torch.matmul(batch_pose1, batch_pose0.permute(0, 2, 1))
            batch_relative_pose = np.matmul(pose1, np.linalg.inv(pose0))
            batch_relative_pose = torch.from_numpy(batch_relative_pose).unsqueeze(0).float().to(device)
            t_err, R_err = relative_pose_error(batch_relative_pose, pre_rot, pre_t, ignore_gt_t_thr=0.0)

            metrics['t_errs'] = metrics['t_errs'] + np.array(t_err.reshape(-1).cpu().detach().numpy()).tolist()
            metrics['R_errs'] = metrics['R_errs'] + np.array(R_err.reshape(-1).cpu().detach().numpy()).tolist()
            metrics['identifiers'].append(pair_name)

    print(f'Acc: {recall_image}/{all_image}')
    val_metrics_4tb = aggregate_metrics(metrics, 5e-4)
    val_metrics_4tb['AP50'] = recall_image / all_image
    logger.info('\n' + pprint.pformat(val_metrics_4tb))

    obj_name = int(name[2:])

    res_table.append([f"{LM_id2name_dict[obj_name]}"] + list(val_metrics_4tb.values()))

2024-01-06 00:41:34.548 | INFO     | __main__:<module>:2 - LINEMOD: 1/13
  0%|          | 0/80 [00:00<?, ?it/s]

100%|██████████| 80/80 [00:01<00:00, 70.13it/s]
2024-01-06 00:41:45.043 | INFO     | __main__:<module>:88 - 
{'AP50': 0.63125,
 'R:ACC15': 0.03333333333333333,
 'R:ACC30': 0.17083333333333334,
 'R:auc@15': 0.010899127788013881,
 'R:auc@30': 0.053468079533841875,
 'R:medianErr': 65.81129455566406,
 't:ACC15': 0.13125,
 't:ACC30': 0.36666666666666664,
 't:auc@15': 0.050399127966827814,
 't:auc@30': 0.1493361373576853,
 't:medianErr': 37.89562225341797}
2024-01-06 00:41:45.044 | INFO     | __main__:<module>:2 - LINEMOD: 2/13


Acc: 303/480


100%|██████████| 61/61 [00:00<00:00, 82.03it/s]
2024-01-06 00:41:49.561 | INFO     | __main__:<module>:88 - 
{'AP50': 0.32786885245901637,
 'R:ACC15': 0.03278688524590164,
 'R:ACC30': 0.12021857923497267,
 'R:auc@15': 0.011905582007597486,
 'R:auc@30': 0.041365558679855155,
 'R:medianErr': 80.86316299438477,
 't:ACC15': 0.12568306010928962,
 't:ACC30': 0.3879781420765027,
 't:auc@15': 0.048798732863966444,
 't:auc@30': 0.1580640145175444,
 't:medianErr': 37.23520469665527}
2024-01-06 00:41:49.561 | INFO     | __main__:<module>:2 - LINEMOD: 3/13


Acc: 120/366


...

Acc: 166/294


100%|██████████| 67/67 [00:00<00:00, 76.06it/s]
2024-01-06 00:42:44.395 | INFO     | __main__:<module>:88 - 
{'AP50': 0.44527363184079605,
 'R:ACC15': 0.01990049751243781,
 'R:ACC30': 0.11691542288557213,
 'R:auc@15': 0.00526581016148303,
 'R:auc@30': 0.03712153596862236,
 'R:medianErr': 80.24980163574219,
 't:ACC15': 0.10696517412935323,
 't:ACC30': 0.34577114427860695,
 't:auc@15': 0.034933946816680043,
 't:auc@30': 0.1322745702950713,
 't:medianErr': 40.84038543701172}
2024-01-06 00:42:44.396 | INFO     | __main__:<module>:2 - LINEMOD: 13/13


Acc: 179/402


100%|██████████| 74/74 [00:00<00:00, 84.32it/s]
2024-01-06 00:42:50.258 | INFO     | __main__:<module>:88 - 
{'AP50': 0.34234234234234234,
 'R:ACC15': 0.02927927927927928,
 'R:ACC30': 0.10810810810810811,
 'R:auc@15': 0.010476198282327737,
 'R:auc@30': 0.035823216058828454,
 'R:medianErr': 71.86189270019531,
 't:ACC15': 0.0990990990990991,
 't:ACC30': 0.32882882882882886,
 't:auc@15': 0.033737520699028495,
 't:auc@30': 0.12567272608702604,
 't:medianErr': 40.43362045288086}


Acc: 152/444


In [8]:
headers = ["Category"] + list(val_metrics_4tb.keys())
all_data = np.array(res_table)[:, 1:].astype(np.float32)
res_table.append(["Avg"] + all_data.mean(0).tolist())
print(tabulate(res_table, headers=headers, tablefmt='fancy_grid'))

╒═════════════╤════════════╤════════════╤════════════╤═══════════╤═══════════════╤════════════╤════════════╤═══════════╤═══════════╤═══════════════╤══════════╕
│ Category    │   R:auc@15 │   R:auc@30 │    R:ACC15 │   R:ACC30 │   R:medianErr │   t:auc@15 │   t:auc@30 │   t:ACC15 │   t:ACC30 │   t:medianErr │     AP50 │
╞═════════════╪════════════╪════════════╪════════════╪═══════════╪═══════════════╪════════════╪════════════╪═══════════╪═══════════╪═══════════════╪══════════╡
│ ape         │ 0.0108991  │  0.0534681 │ 0.0333333  │  0.170833 │       65.8113 │  0.0503991 │   0.149336 │ 0.13125   │  0.366667 │       37.8956 │ 0.63125  │
├─────────────┼────────────┼────────────┼────────────┼───────────┼───────────────┼────────────┼────────────┼───────────┼───────────┼───────────────┼──────────┤
│ benchvise   │ 0.0119056  │  0.0413656 │ 0.0327869  │  0.120219 │       80.8632 │  0.0487987 │   0.158064 │ 0.125683  │  0.387978 │       37.2352 │ 0.327869 │
├─────────────┼────────────┼────────────